# Loop 10 Analysis: Understanding the Gap and Finding a Path Forward

## Current Situation
- Best CV/LB: 70.659959
- Target: 68.919154
- Gap: 1.74 points (2.5%)

## Key Insights from Evaluator
1. Both optimization attempts that showed improvement FAILED on Kaggle due to overlaps
2. Local Shapely validation is NOT sufficient - Kaggle uses stricter SAT-based validation
3. The bbox3-runner kernel has `repair_overlaps_in_place()` function we haven't used
4. Top kernels run for 3 HOURS, we only ran for 3.5 minutes

In [ ]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
import os

# Tree geometry
TX = [0, 0.125, 0.0625, 0.2, 0.1, 0.35, 0.075, 0.075, -0.075, -0.075, -0.35, -0.1, -0.2, -0.0625, -0.125]
TY = [0.8, 0.5, 0.5, 0.25, 0.25, 0, 0, -0.2, -0.2, 0, 0, 0.25, 0.25, 0.5, 0.5]

def parse_value(s):
    if isinstance(s, str) and s.startswith('s'):
        return float(s[1:])
    return float(s)

def create_tree_polygon(x, y, deg):
    angle_rad = np.radians(deg)
    cos_a, sin_a = np.cos(angle_rad), np.sin(angle_rad)
    vertices = [(tx * cos_a - ty * sin_a + x, tx * sin_a + ty * cos_a + y) for tx, ty in zip(TX, TY)]
    return Polygon(vertices)

def compute_score_per_n(df):
    scores = {}
    for n in range(1, 201):
        prefix = f'{n:03d}_'
        trees = df[df['id'].str.startswith(prefix)]
        all_points = []
        for _, row in trees.iterrows():
            x = parse_value(str(row['x']))
            y = parse_value(str(row['y']))
            deg_col = 'deg' if 'deg' in df.columns else 'angle'
            deg = parse_value(str(row[deg_col]))
            poly = create_tree_polygon(x, y, deg)
            all_points.extend(list(poly.exterior.coords))
        all_points = np.array(all_points)
        side = max(all_points.max(axis=0) - all_points.min(axis=0))
        scores[n] = side**2 / n
    return scores

print('Functions defined')

In [ ]:
# Load baseline and compute scores
df_baseline = pd.read_csv('/home/code/external_data/saspav/santa-2025.csv')
baseline_scores = compute_score_per_n(df_baseline)
baseline_total = sum(baseline_scores.values())
print(f'Baseline total: {baseline_total:.6f}')
print(f'Target: 68.919154')
print(f'Gap: {baseline_total - 68.919154:.6f} ({(baseline_total - 68.919154)/68.919154*100:.2f}%)')

In [ ]:
# Analyze which N values have the most room for improvement
# Theoretical minimum is when trees are packed at maximum density
# For a single tree, the bounding box is approximately 0.7 x 1.0 (width x height)

# Calculate efficiency for each N
efficiency = {}
for n, score in baseline_scores.items():
    # Theoretical minimum: if we could pack n trees with no wasted space
    # Each tree has area ~0.35 (rough estimate)
    # Minimum bounding box side would be sqrt(n * 0.35) for perfect packing
    theoretical_min = 0.35  # Rough estimate per tree
    efficiency[n] = score / (theoretical_min * n / n)  # Normalized

# Find N values with worst efficiency (most room for improvement)
worst_n = sorted(efficiency.items(), key=lambda x: x[1], reverse=True)[:20]
print('N values with worst efficiency (most room for improvement):')
for n, eff in worst_n:
    print(f'  N={n}: score={baseline_scores[n]:.6f}, efficiency={eff:.4f}')

In [ ]:
# Calculate how much improvement is needed per N to reach target
target = 68.919154
gap = baseline_total - target

# If we improve each N proportionally
print(f'Total gap: {gap:.6f}')
print(f'Average improvement needed per N: {gap/200:.6f}')
print(f'Percentage improvement needed: {gap/baseline_total*100:.2f}%')

# Which N values contribute most to the score?
contributions = [(n, score, score/baseline_total*100) for n, score in baseline_scores.items()]
contributions.sort(key=lambda x: x[1], reverse=True)
print('\nTop 10 N values by score contribution:')
for n, score, pct in contributions[:10]:
    print(f'  N={n}: score={score:.6f} ({pct:.2f}%)')

In [ ]:
# Check what the bbox3-runner kernel achieves
# The kernel runs for 3 hours with multi-phase optimization
# Let's understand the expected improvement

print('bbox3-runner approach:')
print('- Phase A: 2-minute runs (exploration)')
print('- Phase B: 10-minute runs (refinement)')
print('- Phase C: 20-minute runs (final polish)')
print('- Uses fix_direction rotation optimization')
print('- Uses repair_overlaps_in_place with donor solution')
print()
print('Our approach so far:')
print('- Only ran for 3.5 minutes total')
print('- No overlap repair implemented')
print('- Both optimization attempts failed on Kaggle due to overlaps')

In [ ]:
# Key insight: The gap of 1.74 points is 2.5% improvement needed
# This is achievable with proper optimization

# Let's check if there are any pre-optimized solutions in snapshots
import glob

snapshot_dirs = glob.glob('/home/nonroot/snapshots/santa-2025/*/')
print(f'Found {len(snapshot_dirs)} snapshot directories')

# Check the latest few snapshots
for snap_dir in sorted(snapshot_dirs)[-5:]:
    sub_path = os.path.join(snap_dir, 'submission/submission.csv')
    if os.path.exists(sub_path):
        df = pd.read_csv(sub_path)
        scores = compute_score_per_n(df)
        total = sum(scores.values())
        print(f'{snap_dir}: {total:.6f}')

In [ ]:
# The critical insight is that we need:
# 1. Proper overlap repair using donor solution
# 2. Much longer optimization runs (hours, not minutes)
# 3. Multi-phase approach with escalating timeouts

# Let's check what the bbox3 binary can do
import subprocess

# Check if bbox3 binary exists
bbox3_paths = [
    '/home/code/experiments/009_multiseed_bbox3/bbox3',
    '/home/code/research/kernels/yongsukprasertsuk_santa-2025-best-keeping-bbox3-runner/bbox3',
]

for path in bbox3_paths:
    if os.path.exists(path):
        print(f'Found bbox3 at: {path}')
        # Check if it's executable
        result = subprocess.run(['file', path], capture_output=True, text=True)
        print(f'  Type: {result.stdout.strip()}')

In [ ]:
# Summary of what we need to do:
print('='*60)
print('STRATEGY FOR NEXT EXPERIMENT')
print('='*60)
print()
print('1. IMPLEMENT OVERLAP REPAIR (CRITICAL)')
print('   - Extract repair_overlaps_in_place() from bbox3-runner')
print('   - Use saspav baseline as donor solution')
print('   - Apply after every optimization run')
print()
print('2. RUN BBOX3 FOR MUCH LONGER')
print('   - Current: 3.5 minutes')
print('   - Target: 30-60 minutes minimum')
print('   - Use multi-phase approach from bbox3-runner')
print()
print('3. APPLY FIX_DIRECTION AFTER EACH PHASE')
print('   - Rotation optimization to tighten bounding box')
print('   - Already implemented, just need to integrate')
print()
print('4. SUBMIT TO VERIFY OVERLAP REPAIR WORKS')
print('   - We have 93 submissions remaining')
print('   - Need to verify Kaggle accepts the repaired solution')